In [1]:
import bs4
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [5]:
response=requests.get("http://aws.imd.gov.in:8091/")


In [13]:
soup=BeautifulSoup(response.text,'html5lib')

table = soup.find('table')


None


In [10]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time


In [2]:
import time
import requests
import re
from bs4 import BeautifulSoup

def ref_rev(base_url,AR):
    dic={'AI':2,'BA':2,'ET':2,'EM':2,'QA':2}

    reviews = []
    dates = []
    country=[]

    for i in range(1, dic.get(AR)):
        url = f"{base_url}/page/{i}"
        response = requests.get(url)
        data = response.text
        soup = BeautifulSoup(data, 'html5lib')

        h2_elements = soup.body.find_all('h2', class_="text_header")
        h3_elements = soup.find_all('time')
        h3_country = soup.find_all(class_="text_sub_header userStatusWrapper")
        for tag in h3_country:
            text = tag.get_text()
            country_match = re.search(r'\((.*?)\)', text)
            if country_match:
                country.append(country_match.group(1))



        h2_texts = [h2.text for h2 in h2_elements[0:10]]
        h3_texts = [h3.text for h3 in h3_elements[0:10]]

        reviews.extend(h2_texts)
        dates.extend(h3_texts)


    return dates, reviews,country




In [3]:
dates0,reviews0,country0= ref_rev("https://www.airlinequality.com/airline-reviews/british-airways",'BA')
dates1,reviews1,country1=ref_rev("https://www.airlinequality.com/airline-reviews/air-india",'AI')
dates2,reviews2,country2=ref_rev("https://www.airlinequality.com/airline-reviews/emirates",'EM')
dates3,reviews3,country3=ref_rev("https://www.airlinequality.com/airline-reviews/etihad-airways",'ET')
dates4,reviews4,country4=ref_rev("https://www.airlinequality.com/airline-reviews/qatar-airways",'QA')

In [5]:
Set_={'Date':dates0,'Customer Review':reviews0,'Country':country0}
BA_reviews=pd.DataFrame(Set_)

Set_1={'Date':dates1,'Customer Review':reviews1,'Country':country1}
Air_Ind_reviews=pd.DataFrame(Set_1)

Set_2={'Date':dates2,'Customer Review':reviews2,'Country':country2}
Emirates_reviews=pd.DataFrame(Set_2)

Set_3={'Date':dates3,'Customer Review':reviews3,'Country':country3}
Ethihad_reviews=pd.DataFrame(Set_3)

Set_4={'Date':dates4,'Customer Review':reviews4,'Country':country4}
Qatar_reviews=pd.DataFrame(Set_4)


In [6]:
def date_1(dat):
    dat["Dates"]=pd.to_datetime(dat["Date"],format='mixed')
    dat["Date"]=dat["Dates"]
    dat.drop(columns=["Dates"],inplace=True)
    return dat

data=[Air_Ind_reviews,BA_reviews,Emirates_reviews,Ethihad_reviews,Qatar_reviews]
for df in data:
    df=date_1(df)

In [7]:
import re
def clean(text):
    text = re.sub('[^A-Za-z]+', ' ', str(text))
    return text

# Cleaning the text in the review column
data=[Air_Ind_reviews,BA_reviews,Emirates_reviews,Ethihad_reviews,Qatar_reviews]
for df in data:
    df['Cleaned Reviews']=df['Customer Review'].apply(clean)


In [10]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
pos_dict = {'J':wordnet.ADJ, 'V':wordnet.VERB, 'N':wordnet.NOUN, 'R':wordnet.ADV}
def token_stop_pos(text):
    tags = pos_tag(word_tokenize(text))
    newlist = []
    for word, tag in tags:
        if word.lower() not in set(stopwords.words('english')):
          newlist.append(tuple([word, pos_dict.get(tag[0])]))
    return newlist

for df in data:
    df['POS tagged']=df['Cleaned Reviews'].apply(token_stop_pos)

In [11]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize(pos_data):
    lemma_rew = " "
    for word, pos in pos_data:
     if not pos:
        lemma = word
        lemma_rew = lemma_rew + " " + lemma
     else:
        lemma = wordnet_lemmatizer.lemmatize(word, pos=pos)
        lemma_rew = lemma_rew + " " + lemma
    return lemma_rew

for df in data:
    df['Lemma']=df['POS tagged'].apply(lemmatize)
Emirates_reviews.head()

,Date,Customer Review,Country,Cleaned Reviews,POS tagged,Lemma
0,2024-04-16,“behaviour is totally not acceptable”,India,behaviour is totally not acceptable,"[(behaviour, n), (totally, r), (acceptable, a)]",behaviour totally acceptable
1,2024-04-12,"""I won't be booking Emirates again""",Netherlands,I won t be booking Emirates again,"[(booking, v), (Emirates, n)]",book Emirates
2,2024-04-12,"""the worst I have traveled""",United Kingdom,the worst I have traveled,"[(worst, a), (traveled, v)]",bad travel
3,2024-03-20,"""absolutely unacceptable""",Thailand,absolutely unacceptable,"[(absolutely, r), (unacceptable, a)]",absolutely unacceptable
4,2024-03-17,"""Very disappointed""",United Arab Emirates,Very disappointed,"[(disappointed, a)]",disappointed


In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# function to calculate vader sentiment
def vadersentimentanalysis(review):
    vs = analyzer.polarity_scores(review)
    return vs['compound']
for df in data:
    df['Sentiment'] = df['Lemma'].apply(vadersentimentanalysis)

# function to analyse
def vader_analysis(compound):
    if compound >= 0.4:
        return 'Positive'
    elif compound < 0 :
        return 'Negative'
    else:
        return 'Neutral'
for df in data:
    df['Analysis'] = df['Sentiment'].apply(vader_analysis)


In [13]:
def rat(col):
    if(col=="Positive"):
        score=5
    elif (col=="Neutral"):
        score=2.5
    else:
        score=1
    return score

for df in data:
    df["Rating"]=df["Analysis"].apply(rat)

Qatar_reviews

,Date,Customer Review,Country,Cleaned Reviews,POS tagged,Lemma,Sentiment,Analysis,Rating
0,2024-04-17,“service was fantastic”,Qatar,service was fantastic,"[(service, n), (fantastic, a)]",service fantastic,0.5574,Positive,5.0
1,2024-04-17,"""did not entitle me to lounge access""",Australia,did not entitle me to lounge access,"[(entitle, v), (lounge, v), (access, n)]",entitle lounge access,0.0000,Neutral,2.5
2,2024-04-15,“Flight attendants are polite”,Taiwan,Flight attendants are polite,"[(Flight, n), (attendants, n), (polite, a)]",Flight attendant polite,0.0000,Neutral,2.5
3,2024-04-13,"""long wait for buses""",Kuwait,long wait for buses,"[(long, a), (wait, n), (buses, n)]",long wait bus,0.0000,Neutral,2.5
4,2024-04-10,"""Overall a good flight""",Germany,Overall a good flight,"[(Overall, v), (good, a), (flight, n)]",Overall good flight,0.4404,Positive,5.0
5,2024-04-09,"""did make up for our disappointment""",United Kingdom,did make up for our disappointment,"[(make, v), (disappointment, n)]",make disappointment,-0.5106,Negative,1.0
6,2024-04-08,"""an enjoyable experience""",Kuwait,an enjoyable experience,"[(enjoyable, a), (experience, n)]",enjoyable experience,0.4404,Positive,5.0
7,2024-04-05,"""compensate us for half of the bags value""",Netherlands,compensate us for half of the bags value,"[(compensate, v), (us, None), (half, n), (bags...",compensate us half bag value,0.3400,Neutral,2.5
8,2024-04-03,"""not enough food for the 6h flight""",Germany,not enough food for the h flight,"[(enough, r), (food, n), (h, n), (flight, n)]",enough food h flight,0.0000,Neutral,2.5
9,2024-04-03,"""a great flight""",United Kingdom,a great flight,"[(great, a), (flight, n)]",great flight,0.6249,Positive,5.0


In [14]:
cols=["Date","Country","Customer Review", "Analysis", "Rating"]
Air_india_Rating=Air_Ind_reviews[cols]
BA_Rating=BA_reviews[cols]
Emirates_Rating=Emirates_reviews[cols]
Ethihad_Rating=Ethihad_reviews[cols]
Qatar_Rating=Qatar_reviews[cols]

In [15]:
Qatar_Rating

,Date,Country,Customer Review,Analysis,Rating
0,2024-04-17,Qatar,“service was fantastic”,Positive,5.0
1,2024-04-17,Australia,"""did not entitle me to lounge access""",Neutral,2.5
2,2024-04-15,Taiwan,“Flight attendants are polite”,Neutral,2.5
3,2024-04-13,Kuwait,"""long wait for buses""",Neutral,2.5
4,2024-04-10,Germany,"""Overall a good flight""",Positive,5.0
5,2024-04-09,United Kingdom,"""did make up for our disappointment""",Negative,1.0
6,2024-04-08,Kuwait,"""an enjoyable experience""",Positive,5.0
7,2024-04-05,Netherlands,"""compensate us for half of the bags value""",Neutral,2.5
8,2024-04-03,Germany,"""not enough food for the 6h flight""",Neutral,2.5
9,2024-04-03,United Kingdom,"""a great flight""",Positive,5.0


In [24]:
Air_Ind_Ratings_1=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\6th Sem\\Sentiment Analysis\\Air_Ind_Ratings.csv")
BA_Ratings_1=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\6th Sem\\Sentiment Analysis\\BA_Ratings.csv")
Emirates_Ratings_1=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\6th Sem\\Sentiment Analysis\\Emirates_Ratings.csv")
Ethihad_Ratings_1=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\6th Sem\\Sentiment Analysis\\Ethihad_Ratings.csv")
Qatar_Ratings_1=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\6th Sem\\Sentiment Analysis\\Qatar_Ratings.csv")

In [30]:
Air_Ind_Ratings_1=pd.concat([Air_Ind_Ratings_1,Air_india_Rating])
Air_Ind_Ratings_1=Air_Ind_Ratings_1.drop_duplicates()

BA_Ratings_1=pd.concat([BA_Ratings_1,BA_Rating])
BA_Rating_1=BA_Ratings_1.drop_duplicates()

Emirates_Ratings_1=pd.concat([Emirates_Ratings_1,Emirates_Rating])
Emirates_Ratings_1=Emirates_Ratings_1.drop_duplicates()

Ethihad_Ratings_1=pd.concat([Ethihad_Ratings_1,Ethihad_Rating])
Ethihad_Ratings_1=Ethihad_Ratings_1.drop_duplicates()

Qatar_Ratings_1=pd.concat([Qatar_Ratings_1,Qatar_Rating])
Qatar_Ratings_1=Qatar_Ratings_1.drop_duplicates()
